<a href="https://colab.research.google.com/github/ourlocalpetition/crawling/blob/master/%EC%97%90%EA%B2%8C%EB%A7%90%ED%95%9C%EB%8B%A4_%EC%82%AC%ED%95%98_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Drive Mount

In [ ]:
# drive mount
import os, sys
from google.colab import drive
drive.mount('/content/gdrive')
my_path = '/content/notebooks'
os.symlink('/content/gdrive/Shared drives/우주청/데이터분석/colab_packages', my_path)
sys.path.insert(0, my_path)

In [ ]:
import os, sys
from google.colab import drive
    
drive.mount('/content/gdrive')
sys.path.insert(0, '/content/gdrive/Shared drives/우주청/데이터분석/colab_packages')

#set direction

In [ ]:
congress_dir='/content/gdrive/Shared drives/우주청/데이터분석/구청사이트분석'
os.chdir(congress_dir)

In [ ]:
os.listdir()

# 전처리 함수

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# -*- coding: utf-8 -*-
from konlpy.tag import Okt
from tqdm import tqdm
import re
import pickle
import csv
import swifter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
from collections import Counter

from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pyLDAvis.gensim 
from gensim.models.coherencemodel import CoherenceModel 

dir_font='/content/gdrive/Shared drives/우주청/데이터분석/font/NanumSquareOTF_acL.otf'


In [ ]:
list_no=['민원','개선','사항','주민','관련','다대','주민','요청','첨부파일','아파트','우리','사하','사하구','저희','수고','안녕','대한','사람','구민','양천구','양천구청','서울시','답변','지역','신월동','목동','청장','생각','구청','위해','이행','생각','문제','대해']

In [ ]:
def clean_text(text):
    """
    한글,영문,숫자 남기고 제거한다.
    """
    
    text = re.sub(r'[^ ㄱ-ㅣ가-힣A-Za-z]', '', text) #특수기호 제거, 정규 표현식    

    return text


In [ ]:
def get_nouns(sentence):
    """
    길이가 2이상인 단어중 명사(Noun)만을 반환한다.
    불용어도 제거
    """
    word=[]
    tonkenizer=Okt()
    tagged=tonkenizer.pos(sentence)
    nouns=[s for s,t in tagged if t in['Noun'] and len(s)>1]
    for i in nouns:
        if i not in list_no:
            word.append(i)
    return word


In [ ]:
def tokenize(df):
    df=clean_text(df.replace('\n','').strip())
    df=get_nouns(df)    
    return df


In [ ]:
def del_sentence(list_del,df):
    for d in list_del:
        for num in range(len(df)-1,0,-1):
            if d in df['token_title'][num]:
                df=df.drop(num)
    return df

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """ Compute c_v coherence for various number of topics 
    Parameters:
    ---------- 
    dictionary : Gensim dictionary 
    corpus : Gensim corpus 
    texts : List of input texts 
    limit : Max num of topics 

    Returns: 
    ------- 
    model_list : List of LDA topic models 
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics 
    """ 
    coherence_values=[]
    model_list = [] 
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model) 
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v') 
        coherence_values.append(coherencemodel.get_coherence()) 
    return model_list, coherence_values



In [ ]:
def clean_sentence(text):
    """
    \n삭제
    """
    
    text = re.sub('\n', '', text) #특수기호 제거, 정규 표현식    

    return text

In [ ]:
def find_optimal_number_of_topics(dictionary, corpus, processed_data): 
    limit = 40; 
    start = 2; 
    step = 6; 
    model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=processed_data, start=start, limit=limit, step=step) 
    
    x = range(start, limit, step) 
    plt.plot(x, coherence_values) 
    plt.xlabel("Num Topics") 
    plt.ylabel("Coherence score") 
    plt.legend(("coherence_values"), loc='best') 
    plt.show()


In [ ]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [ ]:
def find_topic(topic,list_st):
    result_list=[]
    for i in topic:
        i=int(i)
        result_list.append(list_st[i])
    return result_list


In [ ]:
def print_cloud(list_token):
    dic_list=[]
    for sentence in list_token:
        for x in sentence:
            dic_list.append(x)
    count=Counter(dic_list)
    words = dict(count.most_common())

    %matplotlib inline 
    import matplotlib 
    from IPython.display import set_matplotlib_formats 

    matplotlib.rc('font',family = 'Malgun Gothic') 

    set_matplotlib_formats('retina') 

    matplotlib.rc('axes',unicode_minus = False)

    from wordcloud import WordCloud
    wc = WordCloud(max_font_size=200, background_color='white', width=1600, height=1000, font_path = dir_font)
    wc.generate(' '.join(words))
    plt.figure(figsize=(20,10), facecolor='k')
    plt.imshow(wc)
    plt.axis('off')
    return words

In [ ]:
def print_100(text):
    start = 0
    end = 100

    while True:
        print(text[start:end],end='\n')

        if len(text[end:]) < 101:
            print(text[end:])
            break
        else:
            start = end + 1
            end += 101

# 데이터처리

In [ ]:
con=pd.read_csv('saha.csv') # 처리할 데이터 입력

In [ ]:
con_real=pd.read_csv('saha.csv') # 처리할 데이터 입력

In [ ]:
con_real['text'][166]

In [ ]:
con['token_text']=con['text'].swifter.apply(lambda x:tokenize(x))
con['token_title']=con['title'].swifter.apply(lambda x:tokenize(x))

In [ ]:
#쓰레기 데이터 삭제

con=del_sentence(['롯데'],con)
con.reset_index(inplace=True)

In [ ]:
con=con.drop(['index','Unnamed: 0'],axis=1)
con

#데이터 토큰화

In [ ]:
tokenized_doc=con['token_text']
tokenized_doc

In [ ]:
token_title=con['token_title']
token_title

In [ ]:
con['text'][3]

# 데이터 시각화

In [ ]:
#분석할 데이터 입력
analy_data=tokenized_doc

In [ ]:

from gensim import corpora
dictionary = corpora.Dictionary(analy_data)
corpus = [dictionary.doc2bow(text) for text in analy_data]

In [ ]:
import gensim
NUM_TOPICS = 7 #20개의 토픽, k=20
num_words=7
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=num_words)
list_st=[]

for topic in topics:
    store_list=[]
    store=topic[1].split('*')
    for i in range(1,1+num_words):
        k=store[i].split('+')[0][1:-1]
        if k[-1]=='"':
            store_list.append(k[0:-1])
        else:
            store_list.append(k)

    list_st.append(store_list)


for i in range(len(list_st)):
    print("topic%d: "%(i),end='')
    print(list_st[i])


In [ ]:
 for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

In [ ]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable['text']=con['text']
topictable['토픽 내용']=find_topic(topictable['가장 비중이 높은 토픽'],list_st)
topictable[:10]

In [ ]:
most_topic_count=topictable['가장 비중이 높은 토픽'].value_counts()

In [ ]:
most_topic_count.sort_index()

In [ ]:
fr_dict={}
for i in range(len(most_topic_count)):
    fr_dict[int(most_topic_count.sort_index().index[i])]=most_topic_count.sort_index().values[i]
fr_dict
  

In [ ]:
plt.bar(fr_dict.keys(),fr_dict.values())
plt.show()

In [ ]:
print_topic_num=3  #출력할 토픽의 수
print_problem_num=5 # 출력할 청원 수
num=0
for i in most_topic_count.index:
    if num>=print_topic_num:
        break
    print(str(i)+'번째 토픽: ',end='')
    for k in list_st[int(i)]:
        print(k+', ',end='')
    print('\n')

    num_1=0
    for k in topictable.values:
        if num_1>=print_problem_num:
            break
        
        if k[1]==i:
            print_100(k[4])
            print('\n')
            num_1+=1
    num+=1



In [ ]:
for k in topictable.values:
    print(k[1])

#word cloud 출력


In [ ]:
words=print_cloud(analy_data)

In [ ]:
print(words)

In [ ]:
num=0
for key,value in words.items():
    print('%s : %d'%(key,value))
    num+=1
    if num==20:
        break

In [ ]:
words.keys()

In [ ]:
df=pd.DataFrame({"word":words.keys(), "count":words.values()}) 

In [ ]:
df

In [ ]:
df.to_csv('word_count_saha.csv',encoding='CP949')

In [ ]:
def find_text(word):
    num_list=[]
    text_list=[]
    word_list=[]
    title_list=[]
    for i in range(len(analy_data)):
        if word in analy_data[i]:
            print('%d번째 글: '%i,end='')
            print_100(con['text'][i])
            print('\n\n')
            word_list.append(word)
            text_list.append(con['text'][i])
            num_list.append(i)
            title_list.append(con['title'][i])
    df=pd.DataFrame({"word":word_list, "title":title_list, "text":text_list}) 
    return df

# 토픽과 단어에 대한 텍스트 찾기

In [ ]:
df=find_text('재개발')
df['text']=df['text'].swifter.apply(lambda x:clean_sentence(x))
df.to_csv('재개발_사하구.csv',encoding='CP949')